In [1]:
import os
import re

import numpy as np
import pandas as pd
from tqdm import tqdm

from glob import glob
from time import sleep
from ast import literal_eval

import requests
from bs4 import BeautifulSoup as sp

In [2]:
os.makedirs('product_links', exist_ok = True)
os.makedirs('data/intermediate', exist_ok = True)
os.makedirs('data/final', exist_ok = True)

In [3]:
def get_soup_obj(URL, pgNo):
    
    r = requests.get(URL + f'?p={pgNo}')
    return sp(r.content, 'lxml')

In [4]:
def get_12_product_links(soup_obj):
    
    product_list = soup_obj.find('ol', {'class' : 'products list items product-items'}).find_all('li')
    return [product.find('a')['href'] for product in product_list]

In [5]:
def get_page_counts(soup_obj):
    
    no_of_items = int(soup_obj.find('div', {'class' : 'toolbar toolbar-products'}).find_all('span', {'class' : 'toolbar-number'})[-1].text)
    return int(round(no_of_items/12, 0))

In [6]:
def get_page_title(soup_obj):
    
    return soup_obj.find('span', {'data-ui-id' : 'page-title-wrapper'}).text

In [7]:
def save_as_dict(name, file):
    
    with open(f'product_links/{name}.txt', 'w') as f:
        f.write(str(file))

In [8]:
def get_product_links(URL_List, filename):
    
    links_dict = {}

    for URL in tqdm(URL_List):

        product_links = []
        soup = get_soup_obj(URL, 1)
        No_pages = get_page_counts(soup)
        page_title = get_page_title(soup)

        product_links += get_12_product_links(soup)
        sleep(2)

        for pg_No in range(2, No_pages+1):

            soup = get_soup_obj(URL, pg_No)
            product_links += get_12_product_links(soup)
            sleep(2)

        links_dict[page_title] = product_links

    save_as_dict(filename, links_dict)

In [9]:
BreakFast_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/brot-broetchen.html',
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/suesse-brotaufstriche.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/bio-brotaufstrich.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/bio-honig.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/bio-kaffee.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/bio-muesli.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/nussmus.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/pflanzendrinks.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/saaten-keime.html']

In [10]:
get_product_links(BreakFast_URL, 'beakfast_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:31<00:00, 10.12s/it]


In [11]:
Superfood_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/superfood-lebensmittel/superfood-beeren.html',
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/superfood-lebensmittel/superfood-nuesse.html',
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/superfood-lebensmittel/superfood-pulver.html']

In [12]:
get_product_links(Superfood_URL, 'superfood_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:15<00:00,  5.06s/it]


In [13]:
Cook_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/geliermittel.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/essig-oel.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/gemuesebruehe.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/gewuerze-kaufen.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/hefeflocken.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/makrobiotik.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/bio-nudeln.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/huelsenfruechte-kaufen.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/salz-pfeffer.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/kochzutaten/bio-fertiggerichte.html']

In [14]:
get_product_links(Cook_URL, 'cook_dict')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:17<00:00,  7.71s/it]


In [15]:
Bake_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/backen/bio-backmischungen.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/backen/backzutaten.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/backen/mehl-online-kaufen.html',
            'https://www.vitalia-reformhaus.de/bio-lebensmittel/backen/suessungsmittel.html']

In [16]:
get_product_links(Bake_URL, 'bake_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:30<00:00,  7.61s/it]


In [17]:
Sweet_n_Salty_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-suessigkeiten/bio-bonbons.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-suessigkeiten/fruchtschnitten.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-suessigkeiten/kekse-waffeln.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-suessigkeiten/bio-snacks.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-suessigkeiten/bio-schokolade.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-suessigkeiten/trockenfruechte-nuesse.html']

In [18]:
get_product_links(Sweet_n_Salty_URL, 'sweet_n_salty_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:44<00:00,  7.39s/it]


In [19]:
Tea_Enjoyment_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-tee/ayurveda-tee.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-tee/fruechte-tee.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-tee/gruener-tee.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-tee/kraeuter-tee.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-tee/schwarzer-tee.html',
                     'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-tee/teemischungen.html']

In [20]:
get_product_links(Tea_Enjoyment_URL, 'tea_enjoyment_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:10<00:00, 11.78s/it]


In [21]:
Beverages_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-getraenke/getraenkepulver.html',
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-getraenke/fruchtsaft.html',
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/bio-getraenke/muttersaft.html']

In [22]:
get_product_links(Beverages_URL, 'beverages_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:25<00:00,  8.62s/it]


In [23]:
Gluten_Free_Foods_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/glutenfreie-lebensmittel/glutenfreies-brot.html',
                         'https://www.vitalia-reformhaus.de/bio-lebensmittel/glutenfreie-lebensmittel/glutenfreie-kekse.html',
                         'https://www.vitalia-reformhaus.de/bio-lebensmittel/glutenfreie-lebensmittel/glutenfreies-mehl.html',
                         'https://www.vitalia-reformhaus.de/bio-lebensmittel/glutenfreie-lebensmittel/glutenfreies-muesli.html',
                         'https://www.vitalia-reformhaus.de/bio-lebensmittel/glutenfreie-lebensmittel/glutenfreie-nudeln.html',
                         'https://www.vitalia-reformhaus.de/bio-lebensmittel/glutenfreie-lebensmittel/glutenfreie-suessigkeiten.html']

In [24]:
get_product_links(Gluten_Free_Foods_URL, 'gluten_free_foods_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:48<00:00,  8.08s/it]


In [25]:
Vegan_Foods_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/vegane-lebensmittel/vegane-marmelade.html',
                   'https://www.vitalia-reformhaus.de/bio-lebensmittel/vegane-lebensmittel/veganer-brotaufstrich.html',
                   'https://www.vitalia-reformhaus.de/bio-lebensmittel/vegane-lebensmittel/vegane-fertiggerichte.html',
                   'https://www.vitalia-reformhaus.de/bio-lebensmittel/vegane-lebensmittel/veganer-fleischersatz.html',
                   'https://www.vitalia-reformhaus.de/bio-lebensmittel/vegane-lebensmittel/vegane-getraenke.html',
                   'https://www.vitalia-reformhaus.de/bio-lebensmittel/vegane-lebensmittel/vegane-grundzutaten.html',
                   'https://www.vitalia-reformhaus.de/bio-lebensmittel/vegane-lebensmittel/milchersatzprodukte.html',
                   'https://www.vitalia-reformhaus.de/bio-lebensmittel/vegane-lebensmittel/veganes-muesli.html',
                   'https://www.vitalia-reformhaus.de/bio-lebensmittel/vegane-lebensmittel/vegane-suessigkeiten.html']

In [26]:
get_product_links(Vegan_Foods_URL, 'vegan_foods_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [02:37<00:00, 17.51s/it]


In [27]:
product_links = sorted(glob('product_links/*.txt'))

major_cat = ['Backen', 'Frühstück', 'Getränke', 'Kochen',
             'Glutenfreie Lebensmittel', 'Superfood', 'Süßes & Salziges', 
             'Tee-Genuss', 'Vegane Lebensmittel']

product_link_df = pd.DataFrame(columns = ['PROD_Section', 'PROD_Category', 'PROD_LINK'])

In [28]:
for cat, link in zip(major_cat, product_links):
    
    with open(link, 'r') as file:
        dicts_ = literal_eval(file.read())
        
        for key, values in dicts_.items():
            for value in values:
                product_link_df.loc[len(product_link_df)] = [cat, key, value]

product_link_df.to_csv('data/intermediate/product_category_nd_links.csv', index = False)

print(f'Shape of the Product link dataset : {product_link_df.shape}')
print(f'Number of unique products present : {product_link_df.PROD_LINK.nunique()}\n')
product_link_df.head()

Shape of the Product link dataset : (1798, 3)
Number of unique products present : 982



,PROD_Section,PROD_Category,PROD_LINK
0,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...
1,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...
2,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...
3,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...
4,Backen,Backmischungen,https://www.vitalia-reformhaus.de/baukhof-schn...


In [29]:
def get_product_soup(URL):

    r = requests.get(URL)
    return sp(r.content, 'lxml')

In [30]:
def get_product_name(soup_obj):
    
    try:
        return soup_obj.find('h1', {'class' : 'page-title'}).text.strip()
    except:
        return np.nan

In [31]:
def get_product_price(soup_obj):
    
    try:
        return soup_obj.find('span', {'class' : 'price'}).text.strip()
    except:
        return np.nan

In [32]:
def get_product_img_url(soup_obj):
    
    try:
        return soup_obj.find('img', {'alt' : 'main product photo'})['src']
    except:
        return np.nan

In [33]:
def get_product_description(soup_obj):
    
    try:
        a_description = []
        attribute_description = soup_obj.find('div', {'class' : 'product attribute description'}).find_all('p')
        for description in attribute_description:
            if 'Rechtlicher Status' in description.text:
                break
            a_description.append(description.text)
        return ' '.join(a_description).strip()
    except:
        return np.nan

In [34]:
def get_product_overview(soup_obj):
    
    try:
        a_overview = []
        attribute_overview = soup.find('div', {'itemprop' : 'description'}).find_all('li')        
        for overview in attribute_overview:
            a_overview.append(overview.text)
        return ', '.join(a_overview).strip()
    except:
        return np.nan

In [35]:
def get_product_brand(soup_obj):
    
    try:
        Flag = False
        attribute_description = soup_obj.find('div', {'class' : 'product attribute description'}).find_all('p')
        for description in attribute_description:    
            if 'Hersteller Anschrift' in description.text:
                Flag = True
            if Flag:
                brand = re.sub(r'<p>', '', str(description).split('<br')[0])
        return brand
    except:
        return np.nan

In [36]:
scrapped_products_df = pd.DataFrame(columns = ['PROD_NAME', 'PROD_PRICE', 'PROD_IMAGE_URL', 'PROD_DESCRIPTION', 'PRO_OVERVIEW', 'PROD_BRAND', 'PROD_LINK'])

for link in tqdm(product_link_df.PROD_LINK.unique()):
    
    soup = get_product_soup(link)
    
    name = get_product_name(soup)
    
    price = get_product_price(soup)

    img_url = get_product_img_url(soup)

    description = get_product_description(soup)

    overview = get_product_overview(soup)

    brand = get_product_brand(soup)
    
    scrapped_products_df.loc[len(scrapped_products_df)] = [name, price, img_url, description, overview, brand, link]
    
    sleep(3)
    
scrapped_products_df.to_csv('data/intermediate/scrapped_product_information.csv', index = False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [1:11:52<00:00,  4.39s/it]


In [37]:
print(f'Shape of the scraped product information dataset : {scrapped_products_df.shape}')
scrapped_products_df.head()

Shape of the scraped product information dataset : (982, 7)


,PROD_NAME,PROD_PRICE,PROD_IMAGE_URL,PROD_DESCRIPTION,PRO_OVERVIEW,PROD_BRAND,PROD_LINK
0,Bauckhof Wunderbrödchen Protein 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,Frisch gebackene Brötchen im Handumdrehen: Was...,"Backmischung für glutenfreie Brötchen, Für 8 B...",Bauck GmbH,https://www.vitalia-reformhaus.de/bauckhof-wun...
1,Bauckhof Wunderbrödchen Hafer 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,Frisch gebackene Brötchen im Handumdrehen: Was...,Backmischung für glutenfreie Brötchen auf Basi...,Bauck GmbH,https://www.vitalia-reformhaus.de/bauckhof-wun...
2,Bauckhof Wunderbrödchen Weltmeister 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,Frisch gebackene Brötchen im Handumdrehen: Was...,Backmischung für glutenfreie Brötchen mit Saat...,Bauck GmbH,https://www.vitalia-reformhaus.de/bauckhof-wun...
3,Bauckhof Wunderbrödchen Original 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,"Für uns als Mühle, mit regionalen Wurzeln und ...",Backmischung für glutenfreie Brötchen mit Saat...,Bauck GmbH,https://www.vitalia-reformhaus.de/bauckhof-wun...
4,Bauckhof Schneller Käsekuchen glutenfrei 485 g,"3,49 €",https://www.vitalia-reformhaus.de/media/catalo...,Der Käsekuchen gelingt auch vegan. Zubereitung...,"Schnell und unkompliziert, In nur einer Stunde...",Bauck GmbH,https://www.vitalia-reformhaus.de/baukhof-schn...


In [38]:
combined_product_df = product_link_df.merge(scrapped_products_df, on = 'PROD_LINK', how = 'left')
print(f'Shape of the combined products information dataset : {combined_product_df.shape}\n')
combined_product_df.head()

Shape of the combined products information dataset : (1798, 9)



,PROD_Section,PROD_Category,PROD_LINK,PROD_NAME,PROD_PRICE,PROD_IMAGE_URL,PROD_DESCRIPTION,PRO_OVERVIEW,PROD_BRAND
0,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...,Bauckhof Wunderbrödchen Protein 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,Frisch gebackene Brötchen im Handumdrehen: Was...,"Backmischung für glutenfreie Brötchen, Für 8 B...",Bauck GmbH
1,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...,Bauckhof Wunderbrödchen Hafer 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,Frisch gebackene Brötchen im Handumdrehen: Was...,Backmischung für glutenfreie Brötchen auf Basi...,Bauck GmbH
2,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...,Bauckhof Wunderbrödchen Weltmeister 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,Frisch gebackene Brötchen im Handumdrehen: Was...,Backmischung für glutenfreie Brötchen mit Saat...,Bauck GmbH
3,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...,Bauckhof Wunderbrödchen Original 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,"Für uns als Mühle, mit regionalen Wurzeln und ...",Backmischung für glutenfreie Brötchen mit Saat...,Bauck GmbH
4,Backen,Backmischungen,https://www.vitalia-reformhaus.de/baukhof-schn...,Bauckhof Schneller Käsekuchen glutenfrei 485 g,"3,49 €",https://www.vitalia-reformhaus.de/media/catalo...,Der Käsekuchen gelingt auch vegan. Zubereitung...,"Schnell und unkompliziert, In nur einer Stunde...",Bauck GmbH


In [39]:
combined_product_df['STORE_ID'] = 303
combined_product_df['STORE_NAME'] = 'VITALIA GmbH'
combined_product_df['PROD_CATEGORY'] = combined_product_df.PROD_Category + ', ' + combined_product_df.PROD_Section

combined_product_df['PROD_RATINGS'] = np.nan
combined_product_df['PROD_REVIEWS'] = np.nan
combined_product_df['PROD_POPULARITY'] = np.nan

In [40]:
column_name_list = ['STORE_ID', 'STORE_NAME', 'PROD_NAME', 'PROD_DESCRIPTION', 'PROD_BRAND',
                    'PROD_IMAGE_URL', 'PROD_CATEGORY', 'PROD_PRICE', 'PROD_RATINGS',
                    'PROD_REVIEWS', 'PROD_POPULARITY', 'PROD_LINK']

In [41]:
print(f'Shape of final combined product dataset : {combined_product_df.shape}\n')
combined_product_df.head()

Shape of final combined product dataset : (1798, 15)



,PROD_Section,PROD_Category,PROD_LINK,PROD_NAME,PROD_PRICE,PROD_IMAGE_URL,PROD_DESCRIPTION,PRO_OVERVIEW,PROD_BRAND,STORE_ID,STORE_NAME,PROD_CATEGORY,PROD_RATINGS,PROD_REVIEWS,PROD_POPULARITY
0,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...,Bauckhof Wunderbrödchen Protein 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,Frisch gebackene Brötchen im Handumdrehen: Was...,"Backmischung für glutenfreie Brötchen, Für 8 B...",Bauck GmbH,303,VITALIA GmbH,"Backmischungen, Backen",NaN,NaN,NaN
1,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...,Bauckhof Wunderbrödchen Hafer 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,Frisch gebackene Brötchen im Handumdrehen: Was...,Backmischung für glutenfreie Brötchen auf Basi...,Bauck GmbH,303,VITALIA GmbH,"Backmischungen, Backen",NaN,NaN,NaN
2,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...,Bauckhof Wunderbrödchen Weltmeister 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,Frisch gebackene Brötchen im Handumdrehen: Was...,Backmischung für glutenfreie Brötchen mit Saat...,Bauck GmbH,303,VITALIA GmbH,"Backmischungen, Backen",NaN,NaN,NaN
3,Backen,Backmischungen,https://www.vitalia-reformhaus.de/bauckhof-wun...,Bauckhof Wunderbrödchen Original 350g,"3,79 €",https://www.vitalia-reformhaus.de/media/catalo...,"Für uns als Mühle, mit regionalen Wurzeln und ...",Backmischung für glutenfreie Brötchen mit Saat...,Bauck GmbH,303,VITALIA GmbH,"Backmischungen, Backen",NaN,NaN,NaN
4,Backen,Backmischungen,https://www.vitalia-reformhaus.de/baukhof-schn...,Bauckhof Schneller Käsekuchen glutenfrei 485 g,"3,49 €",https://www.vitalia-reformhaus.de/media/catalo...,Der Käsekuchen gelingt auch vegan. Zubereitung...,"Schnell und unkompliziert, In nur einer Stunde...",Bauck GmbH,303,VITALIA GmbH,"Backmischungen, Backen",NaN,NaN,NaN


In [42]:
VITALIA_GmbH_Product_Details = combined_product_df[column_name_list]
print(f'Shape of final product dataset from VITALIA GmbH : {VITALIA_GmbH_Product_Details.shape}\n')

VITALIA_GmbH_Product_Details.to_csv('data/final/VITALIA_GmbH_Grocery_Products.csv', index = False)
VITALIA_GmbH_Product_Details.head()

Shape of final product dataset from VITALIA GmbH : (1798, 12)



,STORE_ID,STORE_NAME,PROD_NAME,PROD_DESCRIPTION,PROD_BRAND,PROD_IMAGE_URL,PROD_CATEGORY,PROD_PRICE,PROD_RATINGS,PROD_REVIEWS,PROD_POPULARITY,PROD_LINK
0,303,VITALIA GmbH,Bauckhof Wunderbrödchen Protein 350g,Frisch gebackene Brötchen im Handumdrehen: Was...,Bauck GmbH,https://www.vitalia-reformhaus.de/media/catalo...,"Backmischungen, Backen","3,79 €",NaN,NaN,NaN,https://www.vitalia-reformhaus.de/bauckhof-wun...
1,303,VITALIA GmbH,Bauckhof Wunderbrödchen Hafer 350g,Frisch gebackene Brötchen im Handumdrehen: Was...,Bauck GmbH,https://www.vitalia-reformhaus.de/media/catalo...,"Backmischungen, Backen","3,79 €",NaN,NaN,NaN,https://www.vitalia-reformhaus.de/bauckhof-wun...
2,303,VITALIA GmbH,Bauckhof Wunderbrödchen Weltmeister 350g,Frisch gebackene Brötchen im Handumdrehen: Was...,Bauck GmbH,https://www.vitalia-reformhaus.de/media/catalo...,"Backmischungen, Backen","3,79 €",NaN,NaN,NaN,https://www.vitalia-reformhaus.de/bauckhof-wun...
3,303,VITALIA GmbH,Bauckhof Wunderbrödchen Original 350g,"Für uns als Mühle, mit regionalen Wurzeln und ...",Bauck GmbH,https://www.vitalia-reformhaus.de/media/catalo...,"Backmischungen, Backen","3,79 €",NaN,NaN,NaN,https://www.vitalia-reformhaus.de/bauckhof-wun...
4,303,VITALIA GmbH,Bauckhof Schneller Käsekuchen glutenfrei 485 g,Der Käsekuchen gelingt auch vegan. Zubereitung...,Bauck GmbH,https://www.vitalia-reformhaus.de/media/catalo...,"Backmischungen, Backen","3,49 €",NaN,NaN,NaN,https://www.vitalia-reformhaus.de/baukhof-schn...
